In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.polynomial.legendre import Legendre, leggauss
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.special import iv  # Bessel I_\nu
from scipy.optimize import minimize_scalar, minimize

# =============================================
#      1. 读取 & 拟合历史VIX数据 (Legendre部分)
# =============================================

excel_path_vix = "./VIXdata_from_1990_01_02.xlsx"
data_hist = pd.read_excel(excel_path_vix, header=None)
vix_data = data_hist[0].values
vix_data = vix_data[~np.isnan(vix_data)]
vix_data = vix_data / 100.0  # 缩放到 [0,1]

# 排序 & 构建经验CDF
vix_sorted = np.sort(vix_data)
N_hist = len(vix_sorted)
cdf_values = np.linspace(1/N_hist, 1, N_hist)

h_interp = interp1d(
    cdf_values,
    vix_sorted,
    kind='linear',
    bounds_error=False,
    fill_value=(vix_sorted[0], vix_sorted[-1])
)

def h(u):
    """F^{-1}(u),  u ∈ [0,1]"""
    return h_interp(u)

# 多项式拟合 h(u)
deg = 20
u_train = np.linspace(0,1,1001)
h_train = h(u_train)

coeffs = np.polyfit(u_train, h_train, deg)  # polyfit 返回从最高次到最低次排列的系数
h_poly = np.poly1d(coeffs)

def h_poly_eval(u):
    return h_poly(u)

def tilde_h(x):
    """
    由题意: tilde_h(x) = h( (x+1)/2 ),  x ∈ [-1,1].
    """
    return h_poly_eval((x+1)/2)

def tilde_h1(x, K):
    """ payoff = (tilde_h(x) - K)^+ """
    return np.maximum(tilde_h(x) - K, 0.0)

# Legendre-Gauss 节点
xg, wg = leggauss(200)

def compute_inner_products_h1(K):
    """
    计算 <tilde_h1, P_n> 对所有 n=0..deg, 返回 array of length (deg+1)
    其中 tilde_h1(x)=(tilde_h(x)-K)^+, P_n=Legendre.basis(n)(x).
    """
    inner_products = []
    for n in range(deg+1):
        Pn_vals = Legendre.basis(n)(xg)
        integrand = tilde_h1(xg, K)*Pn_vals
        val = np.sum(integrand*wg)
        inner_products.append(val)
    return np.array(inner_products)

# 假设某个行权价K、以及无风险利率等固定
r = 0.0376


# =============================================
#     2. 3/2 模型函数定义 (定价 + 校准)
# =============================================

def price_vix_call_3_2(V, K, T, alpha, beta, kappa, r):
    """
    Goard & Mazur (2013) 3/2模型: VIX call定价公式
    ------------
    V: 当前VIX
    K: 行权价
    T: 剩余到期(年)
    alpha, beta, kappa: 3/2模型参数
    r: 无风险利率
    返回:
      期权理论价格
    """
    if T <= 1e-10:
        return max(V - K, 0.0)

    p = 1.0 - np.exp(-alpha*T)
    nu = 1.0 - 2.0*beta/(kappa**2)

    factor_out = (
        (2.0*alpha*np.exp(-r*T)) / (kappa**2*p)
        * np.exp(-2.0*alpha*np.exp(-alpha*T) / (kappa**2 * V * p))
        * (V**(-beta/(kappa**2)+0.5))
        * np.exp(alpha*T*(-beta/(kappa**2)+0.5))
    )
    X = 1.0/K

    def integrand(u):
        z = (4.0*alpha*np.sqrt(u)*np.exp(-0.5*alpha*T)/(kappa**2*np.sqrt(V)*p))
        return (
            u**(0.5 - beta/(kappa**2))
            * (1.0/u - K)
            * np.exp(-2.0*alpha*u/(kappa**2*p))
            * iv(nu, z)
        )

    integral_val, _ = quad(integrand, 0.0, 1.0/K, limit=200)
    call_price = factor_out * integral_val
    return max(call_price, 0.0)


def calibrate_3_2_model(df_, K):
    """
    在给定 df_(含 VIX index, Time to expiry, VIX call option price),
    以及行权价 K 上, 校准 3/2模型参数(alpha, beta, kappa).
    ----------
    返回 (alpha_hat, beta_hat, kappa_hat)
    """
    V_vals = df_["VIX index"].values
    T_vals = df_["Time to expiry"].values
    C_mkt  = df_["VIX call option price"].values

    def objective(params):
        alpha_, beta_, kappa_ = params
        sse = 0.0
        for i in range(len(df_)):
            model_price = price_vix_call_3_2(
                V=V_vals[i],
                K=K,
                T=T_vals[i],
                alpha=alpha_,
                beta=beta_,
                kappa=kappa_,
                r=r
            )
            sse += (model_price - C_mkt[i])**2
        return sse

    # 初始猜测 & 边界(可根据实际经验改动)
    x0 = [1.0, -4.0, 2.0]  # (alpha, beta, kappa)
    bnds = [
        (1e-6, 5.0),    # alpha > 0
        (-5.0, -1e-6), # beta < 0
        (1e-6, 5.0)    # kappa>0
    ]
    res = minimize(objective, x0, bounds=bnds, method='L-BFGS-B')
    if res.success:
        alpha_hat, beta_hat, kappa_hat = res.x
        print("[3/2] Calibration succeeded!")
    else:
        alpha_hat, beta_hat, kappa_hat = x0
        print("[3/2] Calibration failed, use fallback.")
    return alpha_hat, beta_hat, kappa_hat


# =============================================
#     3. 读入期权数据 & 做 Legendre + 3/2 模型
# =============================================

excel_path_data = r"D:\VIX_option_data\VIX_option_data\20241224\20241224_call_20.xlsx"
df_full = pd.read_excel(excel_path_data)

# 假设excel中有列: "Date", "VIX index", "VIX call option", "Time to expiry", "y value"
# 若没有 "y value", 你需将 "VIX index" -> [0,1] or [-1,1] 做映射, 或者在Legendre中直接使用 "VIX index" (看你模型假设).
# 这里示例: 我们假设已经有 "y value" 列可用. 若没有, 你可以 df_full["y value"] = your_mapping( df_full["VIX index"] ).


# 3A. 分割数据: 先用(9~11月)来做校准, 再用(12月)来测试
df_full["Date"] = pd.to_datetime(df_full["Date"])

start_nov = pd.to_datetime("2024-09-01")  # 你可以改成 2024-09-02
end_nov   = pd.to_datetime("2024-11-30")

df_nov = df_full[(df_full["Date"] >= start_nov) & (df_full["Date"] <= end_nov)].copy()
df_dec = df_full[df_full["Date"] >= pd.to_datetime("2024-11-01")].copy()

df_nov.reset_index(drop=True, inplace=True)
df_dec.reset_index(drop=True, inplace=True)

# 行权价(假设固定 0.20);  若多Strike, 你可另行循环
K_example = 0.20


# 3B. ====== Legendre模型：先对(9~11)数据校准 'k' ======

# (a) 先计算 <tilde{h}_1, P_n> for this K
inner_products_h1 = compute_inner_products_h1(K_example)

# (b) 构造 Pn(x_i)
T_t_i_nov = df_nov['Time to expiry'].values
x_i_nov   = df_nov['y value'].values
c_i_nov   = df_nov['VIX call option price'].values
N_nov     = len(df_nov)

Pn_matrix_nov = np.zeros((N_nov, deg+1))
for n in range(deg+1):
    Pn_matrix_nov[:, n] = Legendre.basis(n)(x_i_nov)

tilde_nu_nov = np.zeros((N_nov, deg+1))
for n in range(deg+1):
    factor = (2*n+1)/2.0 * inner_products_h1[n]
    tilde_nu_nov[:, n] = factor * Pn_matrix_nov[:, n]

n_array = np.arange(deg+1)

def objective_legendre(k):
    error = 0.0
    for i in range(N_nov):
        exponent = np.exp(-0.5*k*n_array*(n_array+1)* T_t_i_nov[i])
        model_price = np.exp(-r*T_t_i_nov[i])* np.sum(tilde_nu_nov[i,:]*exponent)
        error += (model_price - c_i_nov[i])**2
    return error

res_legendre = minimize_scalar(objective_legendre, bounds=(0.00000001, 0.1), method='bounded')
if res_legendre.success:
    hat_k = res_legendre.x
    print(f"[Legendre] Calibrated k from (9~11) data = {hat_k}")
else:
    hat_k = 0.02
    print("[Legendre] Optimization failed, use fallback = 0.02")


# 3C. ====== 3/2模型：对(9~11)数据校准 (alpha, beta, kappa) ======

alpha_hat, beta_hat, kappa_hat = calibrate_3_2_model(df_nov, K=K_example)
print("3/2 model params:")
print("alpha =", alpha_hat, "beta =", beta_hat, "kappa =", kappa_hat)


# =============================================
#     4. 在 12 月数据上做测试对比 (Legendre vs 3/2)
# =============================================

# 4A. ---- Legendre 定价 in Dec ----
df_dec = df_dec.copy()
N_dec  = len(df_dec)
T_t_i_dec = df_dec['Time to expiry'].values
x_i_dec   = df_dec['y value'].values
c_i_dec   = df_dec['VIX call option price'].values

# 先计算: Pn(x_i_dec)
Pn_matrix_dec = np.zeros((N_dec, deg+1))
for n in range(deg+1):
    Pn_matrix_dec[:, n] = Legendre.basis(n)(x_i_dec)

# tilde_nu_dec = (2n+1)/2 * <tilde_h1,P_n> * P_n(x_i)  对应12月的 x_i
# 但这里 x_i_dec 是一条条不一样, 所以: 
legendre_prices = []
for i in range(N_dec):
    exponent = np.exp(-0.5*hat_k*n_array*(n_array+1)*T_t_i_dec[i])
    # sum over n
    summation = 0.0
    for n in range(deg+1):
        factor = (2*n+1)/2.0 * inner_products_h1[n] * Pn_matrix_dec[i,n]
        summation += factor * exponent[n]
    model_price_legendre = np.exp(-r*T_t_i_dec[i]) * summation
    legendre_prices.append(model_price_legendre)

df_dec['Legendre Price'] = legendre_prices


# 4B. ---- 3/2 定价 in Dec ----
three_half_prices = []
for i in range(N_dec):
    V_ = df_dec.loc[i, 'VIX index']
    T_ = df_dec.loc[i, 'Time to expiry']
    model_price_3_2 = price_vix_call_3_2(V_, K_example, T_, alpha_hat, beta_hat, kappa_hat, r)
    three_half_prices.append(model_price_3_2)

df_dec['3/2 Model Price'] = three_half_prices


# ---- 计算误差 & 打印对比 ----
df_dec['Abs Error (Legendre)'] = np.abs(df_dec['VIX call option price'] - df_dec['Legendre Price'])
df_dec['Rel Error (Legendre)'] = df_dec['Abs Error (Legendre)']/df_dec['VIX call option price']

df_dec['Abs Error (3/2)'] = np.abs(df_dec['VIX call option price'] - df_dec['3/2 Model Price'])
df_dec['Rel Error (3/2)'] = df_dec['Abs Error (3/2)']/df_dec['VIX call option price']

print("\n========== December Results Comparison ==========")
print(df_dec[[
    'Date',
    'VIX index',
    'VIX call option price',
    'Legendre Price',
    '3/2 Model Price',
    'Abs Error (Legendre)',
    'Rel Error (Legendre)',
    'Abs Error (3/2)',
    'Rel Error (3/2)',
    'Time to expiry'
]])


# ---- 简单可视化 ----
plt.figure(figsize=(10,6))
plt.plot(df_dec['Date'], df_dec['VIX call option price'], label='Market Price', marker='o')
plt.plot(df_dec['Date'], df_dec['Legendre Price'], label='Legendre Price', marker='x')
plt.plot(df_dec['Date'], df_dec['3/2 Model Price'], label='3/2 Model Price', marker='s')

plt.xlabel('Date')
plt.ylabel('VIX Call Option Price')
plt.title('December VIX Call: Market vs. Legendre vs. 3/2')
plt.legend()
plt.grid(True)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: './VIXdata_from_1990_01_02.xlsx'